# Training Models


# Linear Regression

**Linear Regression** makes variation by calculating the weighted sum of the input features + the bias term (intercept):

$$
\hat{y} = \theta_{0} + \theta_{1} x_{1} + \theta_{2} x_{2} + ... + \theta_{n} x_{n}
$$



The equation can also be expressed in vectorized form, for simplicity:

$$
\hat{y} = h_{\theta} (x) = \theta \cdot x
$$

Once you have the desired model, you can train it by _adjusting the parameters until they fit the data well_. A common metric for this is the **MSE** (Mean Squared Error).

In the following examples, the **MSE** will be used as the _cost function_:

$$
MSE = \frac{1}{m} \sum (\theta ^{T} x^{(i)} - y^{(i)})^{2}
$$

## The Normal Equation